In [40]:
import pandas as pd
import numpy as np

In [154]:
def data_prep(load):
    if load:
        df = pd.read_csv('readydata.csv')
        return df
    else:
        df = pd.read_spss('r29i_os_54.sav')
        df = df[df['region']=='Челябинск']
        # exclude rows without salary
#         df.dropna(subset=['yj13.2'], axis='rows', inplace=True)
#         df=df.loc[(df['yj13.2'] != 'ЗАТРУДНЯЮСЬ ОТВЕТИТЬ') & (df['yj13.2'] != 'НЕТ ОТВЕТА') & (df['yj13.2'] != 'ОТКАЗ ОТ ОТВЕТА')]
        # drop irrelevant data
        df = df[['yj13.2', 'yh5', 'yj161.3y', 'yh6', 'y_marst', 'yj72.172', 'yl5', 'ym3', 'yj72.18a', 'yj6.2', 'yj4.1', 'yj6', 'yj6.0']]
        df = df.rename(columns={'yj13.2': 'salary',
                                'yh5': 'gender',
                                'yj161.3y':'experience',
                                'yh6':'age',
                                'y_marst':'mariage_status',
                                'yj72.172':'children',
                                'yl5':'health_problems',
                                'ym3':'health_lvl',
                                'yj72.18a':'education',
                                'yj6.2':'workweek',
                                'yj4.1':'work_field',
                                'yj6':'has_subs', # подчиненные
                                'yj6.0':'count_subs'
                                })
        for col in df.columns:
            df[col]=df[col].replace(to_replace='ЗАТРУДНЯЮСЬ ОТВЕТИТЬ', value=np.nan)
            df[col]=df[col].replace(to_replace='ОТКАЗ ОТ ОТВЕТА', value=np.nan)
            df[col]=df[col].replace(to_replace='НЕТ ОТВЕТА', value=np.nan)
            df.fillna(method='ffill', inplace=True)
            df.fillna(method='bfill', inplace=True)
            
        df['gender'] = df['gender'].map({'МУЖСКОЙ':1, 'ЖЕНСКИЙ':2})
        df['experience']=df['experience'].replace(to_replace='НЕ БЫЛО СТАЖА', value=0)
        df['age']=2020-df['age']
        df['health_problems'] = df['health_problems'].map({'Нет':0, 'Да':1})
        df['has_subs'] = df['has_subs'].map({'Нет':0, 'Да':1})
        df['health_lvl'] = df['health_lvl'].map({'Очень хорошее':1, 
                                         'Хорошее':2, 
                                         'Среднее, не хорошее, но и не плохое':3,
                                         'Плохое':4,
                                         'Совсем плохое':5})
        low = ['НАЧАЛЬНАЯ ИЛИ НЕПОЛНАЯ СРЕДНЯЯ ШКОЛА',
                'ПОЛНАЯ СРЕДНЯЯ ШКОЛА',
                'ПРОФЕССИОНАЛЬНЫЕ КУРСЫ ШОФЕРОВ, ТРАКТОРИСТОВ, БУХГАЛТЕРОВ, МАШИНИСТОК И Т.П.',
                'ПТУ, ФЗУ, ФЗО БЕЗ СРЕДНЕГО ОБРАЗОВАНИЯ',
                'ПТУ СО СРЕДНИМ ОБРАЗОВАНИЕМ, ТЕХНИЧЕСКОЕ УЧИЛИЩЕ (среднее проф.образ. - подготовкa квалифицир.рабочих и служащих',
                'ТЕХНИКУМ, МЕДИЦИНСКОЕ, МУЗЫКАЛЬНОЕ, ХУДОЖЕСТВЕННОЕ, ПЕДАГОГ. УЧИЛИЩЕ (среднее проф.образ. - специалист сред.звена)',
                'ИНСТИТУТ, УНИВЕРСИТЕТ, АКАДЕМИЯ - ДИПЛОМ О СРЕДНЕМ ПРОФЕССИОНАЛЬНОМ ОБРАЗОВАНИИ']
        high = ['ИНСТИТУТ, УНИВЕРСИТЕТ, АКАДЕМИЯ - ДИПЛОМ СПЕЦИАЛИСТА',
                'ИНСТИТУТ, УНИВЕРСИТЕТ, АКАДЕМИЯ - ДИПЛОМ БАКАЛАВРА',
                'ИНСТИТУТ, УНИВЕРСИТЕТ, АКАДЕМИЯ - ДИПЛОМ МАГИСТРА',
                'АСПИРАНТУРА БЕЗ ПОЛУЧЕНИЯ УЧЕНОЙ СТЕПЕНИ',
                'ОРДИНАТУРА, ИНТЕРНАТУРА',
                'УЧЕНАЯ СТЕПЕНЬ КАНДИДАТА НАУК',
                'УЧЕНАЯ СТЕПЕНЬ ДОКТОРА НАУК',
                'ОРДИНАТУРА, ИНТЕРНАТУРА']
        not_married = ['Никогда в браке не состояли', 'Разведены и в браке не состоите', 'Bдовец (вдова)']
        married = ['Состоите в зарегистрированном браке', 'Живете вместе, но не зарегистрированы', 'ОФИЦИАЛЬНО ЗАРЕГИСТРИРОВАНЫ, НО ВМЕСТЕ НЕ ПРОЖИВАЮТ']
        df.replace(dict.fromkeys(not_married, 0), inplace=True)
        df.replace(dict.fromkeys(married, 1), inplace=True)
        df.replace(dict.fromkeys(low, 0), inplace=True)
        df.replace(dict.fromkeys(high, 1), inplace=True)
#         cat_columns = ['salary', 'experience', 'children', 'workweek', 'count_subs']
#         df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)
        df.reset_index(inplace=True, drop=True)
        filename = 'readydata.csv'
        df.to_csv(filename, encoding='utf-8')
        return df

load=False
data_prep(load)
load=True
df=data_prep(load)
df.head()

,Unnamed: 0,salary,gender,experience,age,mariage_status,children,health_problems,health_lvl,education,workweek,work_field,has_subs,count_subs
0,0,30000.0,1,25.0,49.0,0,1.0,0,2,0,45.0,ДРУГАЯ ОТРАСЛЬ ТЯЖЕЛОЙ ПРОМЫШЛЕННОСТИ,0,6.0
1,1,30000.0,2,21.0,47.0,1,2.0,0,3,1,40.0,ОПЕРАЦИИ С НЕДВИЖИМОСТЬЮ,1,6.0
2,2,26000.0,1,4.0,26.0,0,2.0,1,2,0,40.0,ДРУГАЯ ОТРАСЛЬ ТЯЖЕЛОЙ ПРОМЫШЛЕННОСТИ,1,5.0
3,3,26000.0,2,25.0,50.0,1,1.0,1,3,0,40.0,ДРУГАЯ ОТРАСЛЬ ТЯЖЕЛОЙ ПРОМЫШЛЕННОСТИ,1,5.0
4,4,40000.0,2,8.0,29.0,1,1.0,0,3,1,40.0,"ТРАНСПОРТ, СВЯЗЬ",0,5.0


In [155]:
# процент средней женской зарплаты от мужской по отраслям
field=df.work_field.unique()
for wf in field:
    m=df[(df.work_field==wf)&(df.gender==1)]['salary'].mean()
    f=df[(df.work_field==wf)&(df.gender==2)]['salary'].mean()
    print(wf, f/m*100)

ДРУГАЯ ОТРАСЛЬ ТЯЖЕЛОЙ ПРОМЫШЛЕННОСТИ 89.11651728553137
ОПЕРАЦИИ С НЕДВИЖИМОСТЬЮ nan
ТРАНСПОРТ, СВЯЗЬ 88.7945937450888
ТОРГОВЛЯ, БЫТОВОЕ ОБСЛУЖИВАНИЕ 99.43570143884892
ОБРАЗОВАНИЕ 126.65214233841684
ЗДРАВООХРАНЕНИЕ 144.70477975632613
НЕФТЕГАЗОВАЯ ПРОМЫШЛЕННОСТЬ nan
ФИНАНСЫ 114.99999999999999
ВОЕННО-ПРОМЫШЛЕННЫЙ КОМПЛЕКС nan
ЛЕГКАЯ, ПИЩЕВАЯ ПРОМЫШЛЕННОСТЬ 70.32154340836013
ЮРИСПРУДЕНЦИЯ 100.0
СТРОИТЕЛЬСТВО 65.27710843373494
ЖИЛИЩНО-КОММУНАЛЬНОЕ ХОЗЯЙСТВО 107.83410138248848
ГРАЖДАНСКОЕ МАШИНОСТРОЕНИЕ 130.43478260869566
НАУКА, КУЛЬТУРА 175.0
СПОРТ, ТУРИЗМ, РАЗВЛЕЧЕНИЯ 100.0
ЭНЕРГЕТИЧЕСКАЯ ПРОМЫШЛЕННОСТЬ 65.23809523809524
IT, ИНФОРМАЦИОННЫЕ ТЕХНОЛОГИИ nan
АРМИЯ, МВД, ОРГАНЫ БЕЗОПАСНОСТИ 124.64788732394365
РЕКЛАМА, МАРКЕТИНГ nan


In [156]:
field=df.work_field.unique()
for wf in field:
    m=df[(df.work_field==wf)&(df.gender==1)]['salary'].count()
    f=df[(df.work_field==wf)&(df.gender==2)]['salary'].count()
    print(wf, m, f)

ДРУГАЯ ОТРАСЛЬ ТЯЖЕЛОЙ ПРОМЫШЛЕННОСТИ 10 11
ОПЕРАЦИИ С НЕДВИЖИМОСТЬЮ 0 3
ТРАНСПОРТ, СВЯЗЬ 20 18
ТОРГОВЛЯ, БЫТОВОЕ ОБСЛУЖИВАНИЕ 23 40
ОБРАЗОВАНИЕ 16 18
ЗДРАВООХРАНЕНИЕ 4 11
НЕФТЕГАЗОВАЯ ПРОМЫШЛЕННОСТЬ 2 0
ФИНАНСЫ 2 5
ВОЕННО-ПРОМЫШЛЕННЫЙ КОМПЛЕКС 0 1
ЛЕГКАЯ, ПИЩЕВАЯ ПРОМЫШЛЕННОСТЬ 9 10
ЮРИСПРУДЕНЦИЯ 1 1
СТРОИТЕЛЬСТВО 9 10
ЖИЛИЩНО-КОММУНАЛЬНОЕ ХОЗЯЙСТВО 3 7
ГРАЖДАНСКОЕ МАШИНОСТРОЕНИЕ 4 2
НАУКА, КУЛЬТУРА 1 2
СПОРТ, ТУРИЗМ, РАЗВЛЕЧЕНИЯ 1 1
ЭНЕРГЕТИЧЕСКАЯ ПРОМЫШЛЕННОСТЬ 3 3
IT, ИНФОРМАЦИОННЫЕ ТЕХНОЛОГИИ 2 0
АРМИЯ, МВД, ОРГАНЫ БЕЗОПАСНОСТИ 9 10
РЕКЛАМА, МАРКЕТИНГ 1 0


In [157]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273 entries, 0 to 272
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       273 non-null    int64  
 1   salary           273 non-null    float64
 2   gender           273 non-null    int64  
 3   experience       273 non-null    float64
 4   age              273 non-null    float64
 5   mariage_status   273 non-null    int64  
 6   children         273 non-null    float64
 7   health_problems  273 non-null    int64  
 8   health_lvl       273 non-null    int64  
 9   education        273 non-null    int64  
 10  workweek         273 non-null    float64
 11  work_field       273 non-null    object 
 12  has_subs         273 non-null    int64  
 13  count_subs       273 non-null    float64
dtypes: float64(6), int64(7), object(1)
memory usage: 30.0+ KB
